In [1]:
# Imports for plotting
import ipympl
%matplotlib widget
import matplotlib.pyplot as plt
# Has side effects allowing for 3D plots
from mpl_toolkits.mplot3d import Axes3D

In [2]:
import numpy as np

In [15]:
from fenics import *

# Results
uvals = []

# Mesh sizes
t, T = 0.0, 10.0   # start and end times
nt = 10            # number of time steps
a, b = -10, 10     # start and end spacial positions
nx = 200           # number of space steps
dt = (T - t)/ nt

# Initial value
alpha = 1
u0 = Expression('1 + alpha*x[0]*x[0]', degree=2, alpha=alpha)

# Define periodic boundary
class PeriodicBoundary(SubDomain):

    # Left boundary is "target domain" G
    def inside(self, x, on_boundary):
        return bool(x[0] < DOLFIN_EPS and x[0] > -DOLFIN_EPS and on_boundary)

    # Map right boundary (H) to left boundary (G)
    def map(self, x, y):
        y[0] = x[0] - 20.0

# Create mesh and define function space
mesh = IntervalMesh(nx, a, b)
V = FunctionSpace(mesh, 'CG', 1, constrained_domain=PeriodicBoundary())

# Define initial value
u_prev = interpolate(u0, V)
uvals.append(u_prev.compute_vertex_values(mesh))

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(0)
F = u*v*dx + dt*dot(grad(u), grad(v))*dx - (u_prev + dt*f)*v*dx
a, L = lhs(F), rhs(F)

# Time-stepping
u = Function(V)
for n in range(nt):
    # Update current time
    t += dt
    # Compute solution
    solve(a == L, u)
    # Save result
    uvals.append(u.compute_vertex_values(mesh))
    # Update for next loop
    u_prev.assign(u)
print('done')

done


In [16]:
uvals = np.array(uvals)
xvals = mesh.coordinates()[:, 0]
tvals = np.linspace(0, T, nt + 1)

In [17]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.scatter(*np.meshgrid(xvals, tvals), uvals)

FigureCanvasNbAgg()